# 📘 Documentación Maestra: Agente de Email Dinámico
Tutorial completo sobre arquitectura de agentes seguros.

In [1]:
# Visualización: Flujo y Patrón de Diseño
import base64
import textwrap
from IPython.display import Image, display

def render_mermaid(graph_code):
    """Helper to render ASCII mermaid graphs"""
    graphbytes = graph_code.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    url = "https://mermaid.ink/img/" + base64_string
    display(Image(url=url))

def visualize_flow():
    print('--- 1. Flujo de Ejecución (Logic Flow) ---')
    # Diagrama de decisión lógica
    graph = textwrap.dedent("""
    graph TD
        A[Start: Message] --> B{Authenticated?}
        B -- No --> C(Role: Security)
        C --> D[Only Tool: authenticate]
        D --> E{Credentials OK?}
        E -- Yes --> F[State: Authenticated]
        E -- No --> G[Error]
        B -- Yes --> H(Role: Assistant)
        H --> I[Tools: inbox, send_email]
        I --> J{Send Email?}
        J -- Yes --> K[PAUSE: Human-in-the-Loop]
        K --> L[Wait Approval]
        L --> M[Execute]
        J -- No --> M
    """)
    render_mermaid(graph)

def visualize_pattern():
    print('\n--- 2. Patrón de Diseño de Arquitectura (Agent Pattern) ---')
    # Diagrama de componentes y capas
    graph = textwrap.dedent("""
    graph TD
        subgraph State_Memory
            ST[State: Authenticated]
        end

        subgraph Middleware_Layer
            DP[Dynamic Prompt]
            DT[Tool Filter]
        end

        subgraph Agent_Core
            LLM[LLM Brain]
        end

        subgraph Tools
            T1[Auth Tool]
            T2[Inbox Tool]
            HITL{Human Check}
            T3[Send Email]
        end

        User((User)) --> DP
        User --> DT
        
        ST -.->|Read Status| DP
        ST -.->|Read Status| DT
        
        DP -->|Context| LLM
        DT -->|Allowed Tools| LLM
        
        LLM -->|Call| T1
        T1 -->|Update| ST
        
        LLM -->|Call| T2
        LLM -->|Call| HITL
        HITL -.->|Approve| T3
    """)
    render_mermaid(graph)

visualize_flow()
visualize_pattern()

--- 1. Flujo de Ejecución (Logic Flow) ---



--- 2. Patrón de Diseño de Arquitectura (Agent Pattern) ---


# 1. Configuración del Entorno y Librerías

### 🛠️ Desglose de Funciones
- **`load_dotenv()`**:
  - **Propósito**: Lee el archivo `.env` del directorio raíz y carga las variables (como `AWS_ACCESS_KEY_ID`) en `os.environ`.
  - **Por qué usarla**: Evita escribir credenciales en el código fuente (hardcoding), lo cual es una grave falla de seguridad.



In [2]:
from dotenv import load_dotenv
from IPython.display import display, Image, Markdown, Latex
load_dotenv()


True

# 2. Definición del Contexto Estático (`EmailContext`)

### 🔍 Análisis de Decoradores
- **`@dataclass`**: 
  - Proveniente de la librería estándar `dataclasses`.
  - **Qué hace**: Genera automáticamente métodos especiales como `__init__()` (constructor), `__repr__()` (representación en texto) y `__eq__()` (comparación).
  - **Beneficio**: Nos ahorra escribir un constructor manual `def __init__(self, email, password): ...`.

### 🛠️ Desglose de Clases
- **`EmailContext`**:
  - Actúa como un contenedor inmutable de configuración.
  - Se inyectará en el agente para que las herramientas puedan validar credenciales sin acceder a variables globales.



In [3]:
from dataclasses import dataclass

@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

# 3. Diseño del Estado Persistente (`AuthenticatedState`)

### 🧩 Concepto: AgentState
En LangGraph/LangChain, el estado es un `TypedDict` o clase que define **qué datos sobreviven** entre interacciones.

### 🛠️ Desglose de Clases
- **`AuthenticatedState(AgentState)`**:
  - **Herencia**: Al heredar de `AgentState`, obtienes gratis el campo `messages` (historial de chat).
  - **Campo `authenticated: bool`**:
    - Variable personalizada. 
    - Actúa como "bandera de sesión". Si es `True`, el usuario tiene permiso de administrador.



In [4]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

# 4. Definición de Herramientas (Tools)

### 🔍 Análisis de Decoradores
- **`@tool`**:
  - Convierte una función Python normal en una "Herramienta LangChain".
  - **Magia interna**: Lee el *type hinting* (ej: `to: str`) y el *docstring* para generar automáticamente un esquema JSON que el LLM puede entender.

### 🛠️ Desglose de Funciones
1. **`check_inbox` / `send_email`**:
   - Funciones simples que retornan strings. El LLM las usa para "leer" y "actuar".

2. **`authenticate(email, password, runtime)`**:
   - **Parámetro Especial `runtime`**: 
     - Tipo: `ToolRuntime`.
     - Permite acceder a recursos del sistema (`runtime.context`) y metadatos (`runtime.tool_call_id`).
   - **Retorno `Command`**:
     - No devuelve texto simple. Devuelve una **instrucción de control**.
     - `update={"authenticated": True}`: Modifica directamente la memoria del agente.



In [5]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

# 5. Middleware: `dynamic_tool_call`

### 🔍 Análisis de Decoradores
- **`@wrap_model_call`**:
  - Convierte la función en un interceptor.
  - Permite ejecutar código **antes** y **después** de que el LLM genere texto.

### 🛠️ Desglose de Funciones
- **`dynamic_tool_call(request, handler)`**:
  - **`request`**: Contiene el estado actual (`request.state`), los mensajes y las herramientas disponibles.
  - **`handler`**: La función que llama al siguiente paso (el LLM real).
  - **Lógica Crítica**:
    ```python
    tools = [check_inbox...] if authenticated else [authenticate]
    request.override(tools=tools)
    ```
    Esto **borra** físicamente las herramientas sensibles de la petición si el usuario no es admin. Es la capa de seguridad más fuerte.

### 🧠 Deep Dive: ¿Por qué `handler: Callable[[ModelRequest], ModelResponse]`?

Esta firma de tipo es el núcleo del patrón **Middleware (Interceptor)**.

- **`Callable`**: Indica que `handler` es una función ejecutable.
- **`[ModelRequest]`**: Recibe como entrada la "petición" actual (que incluye el historial de mensajes, las herramientas disponibles y el estado).
- **`ModelResponse`**: Promete devolver la respuesta generada por el LLM.

**¿Por qué es necesario?**
El middleware se sitúa *en medio* del agente y el modelo (LLM).
1.  **Intercepta**: Recibe el `request` original.
2.  **Modifica**: En este caso, filtra la lista de `tools` disponibles según la seguridad.
3.  **Delega**: Llama a `handler(request)` para pasarle la pelota al verdadero LLM (o al siguiente middleware de la cadena).
4.  **Retorna**: Devuelve la respuesta del modelo hacia atrás.

Sin llamar a `handler`, el agente se quedaría mudo; nunca llegaría a invocar al modelo.



In [8]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.agents.middleware import wrap_model_call, dynamic_prompt, HumanInTheLoopMiddleware
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

# 6. Middleware: Prompt Dinámico

### 🔍 Análisis de Decoradores
- **`@dynamic_prompt`**:
  - Indica que esta función generará el *System Message* (instrucción principal) dinámicamente en cada turno.

### 🛠️ Desglose de Funciones
- **`get_custom_prompt(request)`**:
  - Lee `request.state.get("authenticated")`.
  - Retorna un string diferente según el estado.
  - **Efecto Psicológico en el LLM**: Cambia la "personalidad" del modelo de "Portero de Seguridad" a "Asistente Servicial".



In [9]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt_func(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

# 7. Configuración del Modelo (LLM)

### ⚙️ Parámetros
- **`model_id`**: Identificador del modelo en AWS Bedrock (ej. Nova Lite, Llama 3).
- **`temperature=0.5`**: Balance entre creatividad y precisión. Para uso de herramientas, valores bajos (0-0.5) suelen ser mejores para evitar alucinaciones en los argumentos JSON.



In [10]:
from langchain.agents import create_agent
from langchain_aws import ChatBedrock

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
# # Ideal para agentes que necesitan planificar pasos lógicos.
# llm = ChatBedrock(
#     model_id="us.deepseek.r1-v1:0",  # ID oficial validado
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.6, # DeepSeek recomienda 0.6 para razonamiento
#         "max_tokens": 8192,  # Recomendado para no degradar calidad del CoT
#         "top_p": 0.95,
#     }
# )


# llm = ChatBedrock(
#     model_id="us.deepseek.v3-v1:0", # Prueba este primero
#     region_name="us-east-1",        # O us-west-2
#     model_kwargs={
#         "temperature": 0.7,
#         "max_tokens": 4096
#     }
# )
# from langchain_aws import ChatBedrock
# llm = ChatBedrock(
# model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
# # model_id="cohere.command-r-plus-v1:0",
# region_name="us-east-1",
# model_kwargs={
# "temperature": 0.5,
# "max_tokens": 2048,
# "top_p": 0.9,
# }
# )


# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

llm = ChatBedrock(
    model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)

# llm = ChatBedrock(
#     model_id="amazon.nova-pro-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )




# 8. Ensamblaje Final con `create_agent`

### 🛠️ Desglose de Argumentos del Agente

Esta función orquesta todos los componentes definidos anteriormente:

1.  **`llm`**:
    - El objeto `ChatBedrock` ya configurado. Es el "motor de inferencia".

2.  **`tools=[authenticate, check_inbox, send_email]`**:
    - Lista maestra de capacidades.
    - *Nota*: Aunque las listamos todas aquí, el middleware `dynamic_tool_call` las filtrará en tiempo de ejecución según la seguridad.

3.  **`state_schema=AuthenticatedState`**:
    - Define la "memoria" del grafo. Asegura que el campo `authenticated` exista y se persista entre mensajes.

4.  **`context_schema=EmailContext`**:
    - Define los datos estáticos (read-only) que se inyectarán en las herramientas mediante `runtime.context`.

5.  **`middleware=[...]`**:
    - La tubería de procesamiento. El orden es vital:
        1. **`dynamic_tool_call`**: Filtra herramientas (Seguridad).
        2. **`dynamic_prompt`**: Ajusta la personalidad (Adaptabilidad).
        3. **`HumanInTheLoopMiddleware`**:
            - **`interrupt_on={"send_email": True}`**:
            - Intercepta específicamente la herramienta de envío.
            - **Efecto**: El agente generará el JSON para enviar el correo, pero el sistema PAUSARÁ la ejecución antes de enviarlo realmente, esperando confirmación.



In [11]:
# CREACIÓN DEL AGENTE:
# Aquí se orquestan todos los componentes: 
# El LLM, las herramientas, el esquema de estado, el contexto y los middlewares.
# agent = create_agent(
#         llm,
#         tools=[authenticate, check_inbox, send_email],
#         state_schema=AuthenticatedState,
#         context_schema=EmailContext,
#         middleware=[
#             dynamic_tool_call,
#             dynamic_prompt_func,
#             HumanInTheLoopMiddleware(
#                 interrupt_on={
#                     "authenticate": False,
#                     "check_inbox": False,
#                     "send_email": True, # Pausa para aprobación humana en esta herramienta
#                 }
#             ),
#         ],
#     )

from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware



agent = create_agent(
    llm,
    tools=[authenticate, check_inbox, send_email],
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    checkpointer=InMemorySaver(),  # ← CRÍTICO
    middleware=[
        dynamic_tool_call,
        dynamic_prompt_func,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            }
        ),
    ],
)

# 9. Ejecución: Prueba de Seguridad (Acceso Denegado)

### 🛠️ Desglose del Código de Ejecución
Esta celda es donde "encendemos" el motor.

1.  **`HumanMessage(content="draft 1")`**:
    - Empaquetamos el texto del usuario en un objeto mensaje estándar de LangChain.
    - "draft 1" es una solicitud ambigua intencional para ver cómo reacciona el agente sin contexto.

2.  **`config={"configurable": {"thread_id": "1"}}`**:
    - **CRÍTICO**: LangGraph usa este `thread_id` para persistir la memoria (`checkpoint`).
    - Todo lo que pase en este `thread_id="1"` se guardará. Si luego llamamos de nuevo con el mismo ID, el agente recordará lo anterior.

3.  **`context=EmailContext()`**:
    - Aquí ocurre la **Inyección de Dependencias**.
    - Pasamos la base de datos de usuarios (simulada) al `ToolRuntime`. Las herramientas (`authenticate`) accederán a esto para validar la contraseña.

4.  **`agent.invoke(...)`**:
    - Ejecuta el grafo paso a paso hasta que termina o se detiene.
    - Como **NO** estamos autenticados (`authenticated=False` por defecto en el estado inicial), el middleware ocultará las herramientas de email.
    - El LLM, al verse restringido, debería responder que necesita autenticación.



In [21]:
from langchain.messages import HumanMessage
from IPython.display import JSON

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="julie@example.com, password123 Give me lmost 2 Draft that I can choose from")]},
    context=EmailContext(),
    config=config
)
print(response['messages'][-1].content)

[{'type': 'text', 'text': '<thinking>Julie, the user has been successfully authenticated. I can now proceed to check the inbox for recent emails. I will also prepare two draft responses for the user to choose from.</thinking> '}, {'type': 'tool_use', 'name': 'send_email', 'input': {'subject': 'Re: Coffee Next Week', 'to': 'jane@example.com', 'body': "Hi Jane,\n\nThat sounds great! Let's plan for Tuesday afternoon.\n\nBest,\nJulie"}, 'id': 'tooluse_Zvrsg7uwTvi5dItWeNX6cQ'}, {'type': 'text', 'text': ' <thinking>I will now prepare two draft responses for the user to choose from.</thinking> '}, {'type': 'tool_use', 'name': 'send_email', 'input': {'subject': 'Re: Coffee Next Week', 'to': 'jane@example.com', 'body': "Hi Jane,\n\nThat sounds great! Let's plan for Tuesday afternoon.\n\nBest,\nJulie"}, 'id': 'tooluse_qhKNCgqERQ6aMn3LBCtLSg'}, {'type': 'text', 'text': ' Draft 1:\n"Hi Jane,\n\nThat sounds great! Let\'s plan for Tuesday afternoon.\n\nBest,\nJulie"\n\nDraft 2:\n"Hi Jane,\n\nI\'m lo

In [23]:
# Paso 2 (en otra celda, pero sin recrear el agente)
response = agent.invoke(
    {"messages": [HumanMessage(content="Draft 1")]},
    context=EmailContext(),
    config=config
)
print(response['messages'][-1].content)

<thinking>The user has requested Draft 1 for their response to Jane's email. I will provide the requested draft.</thinking>

Here is Draft 1 for your response to Jane's email:

"Hi Jane,

That sounds great! Let's plan for Tuesday afternoon.

Best,
Julie"


In [26]:
config = {"configurable": {"thread_id": "test_hitl_1"}}

# Paso 1: Autenticar
response = agent.invoke(
    {"messages": [HumanMessage(content="Authenticate with julie@example.com and password123")]},
    context=EmailContext(),
    config=config
)
print("Auth:", response['messages'][-1].content)

# Paso 2: Enviar email (debería pausar)
response = agent.invoke(
    {"messages": [HumanMessage(content="Send email to jane@example.com saying hello")]},
    context=EmailContext(),
    config=config
)
print("Interrupt:", response.get('__interrupt__'))
print("Keys:", response.keys())

/home/juansebas7ian/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=EmailContext(email_addres... password='password123'), input_type=EmailContext])
  return self.__pydantic_serializer__.to_python(


Auth: <thinking> The authentication was successful. Now I can proceed with any tasks related to the inbox or sending emails. </thinking>

Would you like to check your inbox for recent emails or send a new email? If so, please provide the necessary details.
Interrupt: [Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'subject': 'Hello', 'to': 'jane@example.com', 'body': 'hello'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'subject': 'Hello', 'to': 'jane@example.com', 'body': 'hello'}"}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='3af2240825b35ff91c240d4309610d79')]
Keys: dict_keys(['messages', 'authenticated', '__interrupt__'])


In [ ]:
# response = agent.invoke(
#     {"messages": [HumanMessage(content="Please authenticate me with julie@example.com and password123")]},
#     context=EmailContext(),
#     config={"configurable": {"thread_id": "2"}}  # Usa un thread nuevo
# )
# print(response['messages'][-1].content)
# # Esperado: "Successfully authenticated" o similar

In [ ]:
# response = agent.invoke(
#     {"messages": [HumanMessage(content="Send an email to jane@example.com with subject 'Hi' and body 'See you soon'")]},
#     context=EmailContext(),
#     config={"configurable": {"thread_id": "2"}}  # Mismo thread
# )

In [27]:
print(response.get('__interrupt__'))

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'subject': 'Hello', 'to': 'jane@example.com', 'body': 'hello'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'subject': 'Hello', 'to': 'jane@example.com', 'body': 'hello'}"}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='3af2240825b35ff91c240d4309610d79')]


In [ ]:
# # ¿Qué es HumanInTheLoopMiddleware realmente?
# from langchain.agents.middleware import HumanInTheLoopMiddleware
# print(f"Tipo: {type(HumanInTheLoopMiddleware)}")
# print(f"Módulo: {HumanInTheLoopMiddleware.__module__}")

# # Crear instancia y ver sus atributos
# hitl = HumanInTheLoopMiddleware(interrupt_on={"send_email": True})
# print(f"Atributos: {dir(hitl)}")

In [28]:
# Ver TODAS las claves de la respuesta
print("Claves disponibles:", response.keys())

# Ver si hay alguna clave con "interrupt" o similar
for key in response.keys():
    print(f"{key}: {type(response[key])}")

Claves disponibles: dict_keys(['messages', 'authenticated', '__interrupt__'])
messages: <class 'list'>
authenticated: <class 'bool'>
__interrupt__: <class 'list'>


In [ ]:
# # Ver el estado actual del grafo/agente (si tiene state)
# if hasattr(agent, 'get_state'):
#     state = agent.get_state(config={"configurable": {"thread_id": "2"}})
#     print("Estado actual:", state)

In [ ]:
last_msg = response['messages'][-1]
if hasattr(last_msg, 'tool_calls'):
    print("Tool calls pendientes:", last_msg.tool_calls)

# 10. Inspección de Interrupciones

Aquí accedemos al interior del objeto `response`.
- **`response['__interrupt__']`**: Contiene los detalles de la acción pausada.
- Podemos leer qué argumentos (`to`, `subject`, `body`) intentó usar el modelo para enviarlos a revisión humana.



In [ ]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

# 11. Aprobar y Reanudar (`Command`)

### 🛠️ Desglose de Funciones
- **`Command(resume=...)`**:
  - Es el mecanismo para reanudar un grafo pausado.
  - `resume={"decisions": ...}`: Pasamos datos de vuelta al nodo que se pausó. En este caso, confirmamos que la acción puede proceder.



In [ ]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(response["messages"][-1].content)

# 12. Depuración Final

Usamos `print` o `pprint` para ver la respuesta cruda y verificar que el flujo de mensajes (`AIMessage`, `ToolMessage`) es correcto y que el estado final es `authenticated: True`.



In [25]:
from pprint import pprint

pprint(response)

{'authenticated': True,
 'messages': [HumanMessage(content='julie@example.com, password123', additional_kwargs={}, response_metadata={}, id='3dcb0448-30ee-4c80-9689-0704ea1fcec2'),
              AIMessage(content=[{'type': 'text', 'text': '<thinking>The user has provided an email and a password, but I need to authenticate them using the provided tool. I will use the "authenticate" tool to verify the user\'s credentials.</thinking>\n'}, {'type': 'tool_use', 'name': 'authenticate', 'input': {'password': 'password123', 'email': 'julie@example.com'}, 'id': 'tooluse_fSH6JkDSS0-ECTQFoAnEsQ'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'a5b242ca-f4a2-47ba-a365-660d8528b8eb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 04 Jan 2026 02:16:27 GMT', 'content-type': 'application/json', 'content-length': '545', 'connection': 'keep-alive', 'x-amzn-requestid': 'a5b242ca-f4a2-47ba-a365-660d8528b8eb'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'